Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/deployment/tensorflow/tensorflow-model-register-and-deploy.png)

# Register TensorFlow SavedModel and deploy as webservice

Following this notebook, you will:

 - Learn how to register a TF SavedModel in your Azure Machine Learning Workspace.
 - Deploy your model as a web service in an Azure Container Instance.

## Prerequisites

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the [configuration notebook](../../../configuration.ipynb) to install the Azure Machine Learning Python SDK and create a workspace.

In [ ]:
import azureml.core

# Check core SDK version number.
print('SDK version:', azureml.core.VERSION)

## Initialize workspace

Create a [Workspace](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace%28class%29?view=azure-ml-py) object from your persisted configuration.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

### Download the Model

Download and extract the model from https://amlsamplenotebooksdata.blob.core.windows.net/data/flowers_model.tar.gz to "models" directory

In [ ]:
import os
import tarfile
import urllib.request

# create directory for model
model_dir = 'models'
if not os.path.isdir(model_dir):
    os.mkdir(model_dir)

url="https://amlsamplenotebooksdata.blob.core.windows.net/data/flowers_model.tar.gz"
response = urllib.request.urlretrieve(url, model_dir + "/flowers_model.tar.gz")
tar = tarfile.open(model_dir + "/flowers_model.tar.gz", "r:gz")
tar.extractall(model_dir)

## Register model

Register a file or folder as a model by calling [Model.register()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py#register-workspace--model-path--model-name--tags-none--properties-none--description-none--datasets-none--model-framework-none--model-framework-version-none--child-paths-none-). For this example, we have provided a TensorFlow SavedModel (`flowers_model` in the notebook's directory).

In addition to the content of the model file itself, your registered model will also store model metadata -- model description, tags, and framework information -- that will be useful when managing and deploying models in your workspace. Using tags, for instance, you can categorize your models and apply filters when listing models in your workspace. Also, marking this model with the scikit-learn framework will simplify deploying it as a web service, as we'll see later.

In [ ]:
from azureml.core import Model

model = Model.register(workspace=ws,
                       model_name='flowers',                        # Name of the registered model in your workspace.
                       model_path= model_dir + '/flowers_model',    # Local Tensorflow SavedModel folder to upload and register as a model.
                       model_framework=Model.Framework.TENSORFLOW,  # Framework used to create the model.
                       model_framework_version='1.14.0',            # Version of Tensorflow used to create the model.
                       description='Flowers model')

print('Name:', model.name)

## Deploy model

Deploy your model as a web service using [Model.deploy()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py#deploy-workspace--name--models--inference-config--deployment-config-none--deployment-target-none-). Web services take one or more models, load them in an environment, and run them on one of several supported deployment targets.

For this example, we will deploy your TensorFlow SavedModel to an Azure Container Instance (ACI).

### Use a default environment (for supported models)

The Azure Machine Learning service provides a default environment for supported model frameworks, including TensorFlow, based on the metadata you provided when registering your model. This is the easiest way to deploy your model.

**Note**: This step can take several minutes.

In [ ]:
from azureml.core import Webservice
from azureml.exceptions import WebserviceException

service_name = 'tensorflow-flower-service'

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

service = Model.deploy(ws, service_name, [model])
service.wait_for_deployment(show_output=True)

After your model is deployed, perform a call to the web service.

In [ ]:
import requests

headers = {'Content-Type': 'application/json'}

if service.auth_enabled:
    headers['Authorization'] = 'Bearer '+ service.get_keys()[0]
elif service.token_auth_enabled:
    headers['Authorization'] = 'Bearer '+ service.get_token()[0]

scoring_uri = service.scoring_uri # If you have a SavedModel with classify and regress, 
                                  #  you can change the scoring_uri from 'uri:predict' to 'uri:classify' or 'uri:regress'.
print(scoring_uri)

with open('tensorflow-flower-predict-input.json', 'rb') as data_file:
    response = requests.post(
        scoring_uri, data=data_file, headers=headers)
print(response.status_code)
print(response.elapsed)
print(response.json())

When you are finished testing your service, clean up the deployment.

In [ ]:
service.delete()